In [11]:
import numpy as np
import pandas as pd
import math
import inspect
from dataclasses import dataclass
from typing import List, Callable, Dict, Any

np.seterr(all="raise")  # Error on overflow


@dataclass
class FoundationParameters:
    REO_DENSITY: float = 7850  # kg/m3
    LOAD_DEAD: float = 8000  # kN
    LOAD_LIVE: float = 2500  # kN
    BEARINGPRESSURE_ALLOWABLE: float = 500  # kPa
    FTG_COVER: float = 0.060  # m
    COLUMN_WIDTH: float = 0.500  # m


class CalculationStep:
    def __init__(self, name: str, func: Callable, dependencies: List[str]):
        self.name = name
        self.func = func
        self.dependencies = dependencies


class FoundationCalculator:
    def __init__(self, params: FoundationParameters):
        self.params = params
        self.calculations: List[CalculationStep] = []

    def add_calculation(self, name: str, func: Callable):
        dependencies = list(inspect.signature(func).parameters.keys())
        self.calculations.append(CalculationStep(name, func, dependencies))

    def calculate(self, sizes: np.ndarray) -> np.ndarray:
        results: Dict[str, Any] = {col: sizes[col] for col in sizes.dtype.names}
        new_fields = []

        for step in self.calculations:
            args = {dep: results[dep] for dep in step.dependencies}
            result = step.func(**args)
            results[step.name] = result

            if step.name not in sizes.dtype.names:
                new_fields.append((step.name, result.dtype))

        if new_fields:
            new_dtype = sizes.dtype.descr + new_fields
            new_sizes = np.empty(sizes.shape, dtype=new_dtype)

            for name in sizes.dtype.names:
                new_sizes[name] = sizes[name]

            for name, _ in new_fields:
                new_sizes[name] = results[name]
        else:
            new_sizes = sizes.copy()

        for step in self.calculations:
            new_sizes[step.name] = results[step.name]

        return new_sizes


class FoundationSizer:
    def __init__(self, params: FoundationParameters, calculator: FoundationCalculator):
        self.params = params
        self.calculator = calculator
        self.sizes = self.generate_foundation_sizes()
        self.calculate_all_properties()
        self.column_names = list(self.sizes.dtype.names)

    def generate_foundation_sizes(self) -> np.ndarray:
        FTG_LEN_MIN = (
            math.ceil(
                math.sqrt(
                    (self.params.LOAD_DEAD + self.params.LOAD_LIVE)
                    / self.params.BEARINGPRESSURE_ALLOWABLE
                )
                / 0.05
            )
            * 0.05
        )
        FTG_LEN_MAX = FTG_LEN_MIN + 1
        FTG_LENS = np.round(
            np.arange(FTG_LEN_MIN, FTG_LEN_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_DPTH_MIN = (
            math.ceil(
                (
                    4
                    * math.sqrt(3570)
                    * math.sqrt(
                        (4760 * self.params.COLUMN_WIDTH**2) / 3
                        + (1 + (3 * self.params.BEARINGPRESSURE_ALLOWABLE) / 19040)
                        * (self.params.LOAD_DEAD + self.params.LOAD_LIVE)
                    )
                    - (9520 + 3 * self.params.BEARINGPRESSURE_ALLOWABLE)
                    * self.params.COLUMN_WIDTH
                )
                / (19040 + 3 * self.params.BEARINGPRESSURE_ALLOWABLE)
                / 0.05
            )
            * 0.05
        )
        FTG_DPTH_MAX = FTG_DPTH_MIN * 2
        FTG_DPTHS = np.round(
            np.arange(FTG_DPTH_MIN, FTG_DPTH_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_CONC_STRENGTHS = np.array([20, 25, 32, 40, 50, 65], dtype=np.float32)

        FTG_REO_SIZES = np.round(
            np.array(
                [0.012, 0.016, 0.02, 0.024, 0.028, 0.032, 0.036, 0.04],
                dtype=np.float32,
            ),
            3,
        )

        FTG_REO_CTS = np.unique(
            np.round(
                np.concatenate(
                    [
                        np.arange(0.1, 0.301, 0.025, dtype=np.float32),
                        np.arange(0.08, 0.301, 0.02, dtype=np.float32),
                    ]
                ),
                3,
            )
        )

        foundation_sizes = np.array(
            np.meshgrid(
                FTG_LENS, FTG_DPTHS, FTG_CONC_STRENGTHS, FTG_REO_SIZES, FTG_REO_CTS
            )
        ).T.reshape(-1, 5)

        return np.rec.array(
            foundation_sizes,
            dtype=[
                ("FtgLength", "f4"),
                ("FtgDepth", "f4"),
                ("fc", "f4"),
                ("ReoSize", "f4"),
                ("ReoCts", "f4"),
            ],
        )

    def calculate_all_properties(self) -> None:
        self.sizes = self.calculator.calculate(self.sizes)

    def remove_fails(self) -> None:
        valid_mask = (
            (self.sizes["Mrat"] <= 1)
            & (self.sizes["VLrat"] <= 1)
            & (self.sizes["VPrat"] <= 1)
            & (self.sizes["Astact"] >= self.sizes["Ast"])
            & (self.sizes["BPrat"] <= 1)
        )
        self.sizes = self.sizes[valid_mask]

    def filter_match(self, **kwargs) -> None:
        mask = np.ones(len(self.sizes), dtype=bool)
        for key, value in kwargs.items():
            if key in self.column_names:
                mask &= self.sizes[key] == value
        self.sizes = self.sizes[mask]

    def sort_by_cost(self) -> None:
        self.sizes = np.sort(self.sizes, order="Cost")

    def print_foundation_sizes(self, num_rows: int = 6) -> None:
        if len(self.sizes) == 0:
            print("No foundation sizes available.")
            return

        pd.set_option("display.float_format", lambda x: "%.3f" % x)
        df = pd.DataFrame(self.sizes).reset_index(names=["Row"])

        total_rows = df.shape[0]
        step = max(1, total_rows // (num_rows - 1))

        selected_rows = np.arange(0, total_rows, step)
        if len(selected_rows) < num_rows:
            selected_rows = np.append(selected_rows, total_rows - 1)
        elif len(selected_rows) > num_rows:
            selected_rows = selected_rows[:num_rows]

        df_selected = df.iloc[selected_rows]
        print(df_selected.to_string(index=False))

    def print_first_foundation_sizes(self, num_rows: int = 6):
        if len(self.sizes) == 0:
            print("No foundation sizes available.")
            return

        pd.set_option("display.float_format", lambda x: "%.3f" % x)
        df = pd.DataFrame(self.sizes).reset_index(names=["Row"])

        df_head = df.head(num_rows)
        print(df_head.to_string(index=False))

    def print_foundation_details(self, row_index: int) -> None:
        if row_index < 0 or row_index >= len(self.sizes):
            print(f"Row index {row_index} is out of range.")
            return

        df = pd.DataFrame(self.sizes)
        row = df.iloc[row_index]

        max_key_length = max(len(key) for key in row.index)

        print(f"Details for foundation size at row {row_index}:")
        for key, value in row.items():
            formatted_value = f"{value:.3f}" if isinstance(value, float) else str(value)
            print(f"{key:<{max_key_length}} : {formatted_value}")


# Usage
params = FoundationParameters()
calculator = FoundationCalculator(params)

# Add calculations
calculator.add_calculation(
    "Pult",
    lambda FtgDepth, FtgLength: (
        1.2 * (6 * FtgDepth * FtgLength**2 + params.LOAD_DEAD) + 1.5 * params.LOAD_LIVE
    ),
)

calculator.add_calculation(
    "BPmax",
    lambda FtgDepth, FtgLength: (
        6 * FtgDepth * FtgLength**2 + params.LOAD_LIVE + params.LOAD_DEAD
    )
    / (FtgLength**2),
)

calculator.add_calculation(
    "BPrat", lambda BPmax: BPmax / params.BEARINGPRESSURE_ALLOWABLE
)

calculator.add_calculation(
    "Dom", lambda FtgDepth, ReoSize: FtgDepth - params.FTG_COVER - ReoSize / 2
)

calculator.add_calculation("BPult", lambda Pult, FtgLength: Pult / FtgLength**2)

calculator.add_calculation(
    "CLR", lambda BPult, Dom: BPult * (params.COLUMN_WIDTH + Dom) ** 2
)

calculator.add_calculation("VPult", lambda Pult, CLR: Pult - CLR)

calculator.add_calculation(
    "fVP",
    lambda ReoSize, Dom, fc: (
        952 * (ReoSize - Dom) * (ReoSize - Dom - params.COLUMN_WIDTH) * np.sqrt(fc)
    ),
)

calculator.add_calculation("VPrat", lambda VPult, fVP: VPult / fVP)

calculator.add_calculation(
    "dv", lambda Dom, FtgDepth: np.maximum(0.9 * Dom, 0.72 * FtgDepth)
)

calculator.add_calculation(
    "VLult",
    lambda BPult, dv, FtgLength: -0.5
    * BPult
    * (params.COLUMN_WIDTH + 2 * dv - FtgLength),
)

calculator.add_calculation("kvo", lambda dv: 2 / (10 + 13 * dv))

calculator.add_calculation("kv", lambda kvo: np.minimum(kvo, 0.15))

calculator.add_calculation(
    "ks", lambda FtgDepth: np.maximum(0.5, (10 / 7) * (1 - FtgDepth))
)

calculator.add_calculation(
    "fVuc", lambda dv, fc, ks, kv: 700 * dv * np.sqrt(fc) * ks * kv
)

calculator.add_calculation("VLrat", lambda VLult, fVuc: VLult / fVuc)

calculator.add_calculation(
    "Mult",
    lambda BPult, FtgLength: (BPult * (7 * params.COLUMN_WIDTH - 10 * FtgLength) ** 2)
    / 800,
)

calculator.add_calculation("Astshr", lambda Mult, Dom: (5 * Mult) / (2 * Dom))

calculator.add_calculation(
    "Astmin", lambda FtgDepth, fc, Dom: (228 * FtgDepth**2 * np.sqrt(fc)) / Dom
)

calculator.add_calculation("Ast", lambda Astmin, Astshr: np.maximum(Astmin, Astshr))

calculator.add_calculation("alpha", lambda fc: 0.85 - 0.0015 * fc)

calculator.add_calculation("gamma", lambda fc: 0.97 - 0.0025 * fc)

calculator.add_calculation(
    "ku",
    lambda Ast, alpha, Dom, fc, gamma, FtgLength: Ast
    / (2000 * alpha * Dom * fc * gamma * FtgLength),
)

calculator.add_calculation(
    "phi", lambda ku: np.minimum(0.85, np.maximum(0.65, 1.24 - 1.08333 * ku))
)

calculator.add_calculation(
    "Astact", lambda ReoSize, ReoCts: (250000 * ReoSize**2 * np.pi) / ReoCts
)

calculator.add_calculation(
    "fMuo",
    lambda Astact, Dom, phi, alpha, fc, FtgLength: (Astact * Dom * phi) / 2
    - (Astact**2 * phi) / (8000 * alpha * fc * FtgLength),
)

calculator.add_calculation("Mrat", lambda Mult, fMuo: Mult / fMuo)

calculator.add_calculation(
    "Cost",
    lambda Astact, FtgLength, FtgDepth, fc: (
        Astact / 1000000 * FtgLength * params.REO_DENSITY * 2 * 3400
        + FtgLength**2 * FtgDepth * (130.866 * np.exp(fc * 0.0111) + 45 + 130)
        + 4 * FtgDepth * FtgLength * 180
    ),
)

# Initialize FoundationSizer and perform calculations
foundation = FoundationSizer(params, calculator)

# Remove fails, sort by cost, and print results
foundation.remove_fails()
foundation.sort_by_cost()
# foundation.print_first_foundation_sizes()
# foundation.print_foundation_sizes()
foundation.print_foundation_details(0)

Details for foundation size at row 0:
FtgLength : 4.650
FtgDepth  : 2.000
fc        : 40.000
ReoSize   : 0.024
ReoCts    : 0.150
Pult      : 13661.364
BPmax     : 497.605
BPrat     : 0.995
Dom       : 1.928
BPult     : 631.812
CLR       : 3724.650
VPult     : 9936.714
fVP       : 27559.312
VPrat     : 0.361
dv        : 1.735
VLult     : 214.690
kvo       : 0.061
kv        : 0.061
ks        : 0.500
fVuc      : 235.953
VLrat     : 0.910
Mult      : 1460.276
Astshr    : 1893.512
Astmin    : 2991.698
Ast       : 2991.698
alpha     : 0.790
gamma     : 0.870
ku        : 0.006
phi       : 0.850
Astact    : 3015.929
fMuo      : 2464.675
Mrat      : 0.592
Cost      : 771691.125
